In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Utility Functions

In [3]:
def load_cifar10_dataset():
    """
    Download and pre-process the CIFAR10 dataset.

    Returns:
        a "loader" for our training data, a loader for our
        test data, a tuple of classes
    """

    # Create a "transform" that will allow us to transform the
    # image objects to tensors and normalize their values. Find
    # out more about PyTorch transform here:
    # https://pytorch.org/docs/stable/torchvision/transforms.html
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    # Download the traning dataset to our data folder and create a
    # data loader that will allow us to load in batches of the
    # images. torch.utils.data.DataLoader is an iterator which lets us
    # batch the data, shuffle the data, and load the data in parallel
    #using multiprocessing workers.
    trainset = torchvision.datasets.CIFAR10(root='../data', train=True,
                                        download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

    # Download the test data and create another data loader.
    testset = torchvision.datasets.CIFAR10(root='../data', train=False,
                                       download=True, transform=transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

    # Create a tuple of classes contained in the data set.
    classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

    return trainloader, testloader, classes

# Model and Training Definition

This is very similar to what we had in the lesson 3 example, but note the 2 very important changes that map the training operations to the GPU.

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # A 1st convolutional layer.
        self.conv1 = nn.Conv2d(3, 6, 5)

        # A "pooling" layer. Find out more about these here:
        # http://cs231n.github.io/convolutional-networks/
        self.pool = nn.MaxPool2d(2, 2)

        # A 2nd convolutional layer.
        self.conv2 = nn.Conv2d(6, 16, 5)

        # A fully connected linear layer like we have seen before.
        self.fc1 = nn.Linear(16 * 5 * 5, 120)

        # Another fully connected linear layer.
        self.fc2 = nn.Linear(120, 84)

        # A final fully connected linear layer.
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

def nn_fit(trainloader, learning_rate, epochs, device):
    """
    Train a CNN using SGD and pytorch.

    Args:
        trainloader - a training data set loader
        learning_rate - learning rate used in SGD
        epochs - number of epochs for the SGD loop
    Returns:
        The trained model
    """

    # initialize the model
    model = Net()
    
    # NOTE - THIS IMPORTANT CHANGE
    # Run our model training operations of the GPU.
    model.to(device)

    # our error/loss function
    criterion = nn.CrossEntropyLoss()

    # define our SGD optimizer
    optimiser = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

    # loop over our epochs, similar to our previous implementations
    for epoch in range(epochs):

        # This time we will keep a running total of our
        # loss so we can print out some stats.
        running_loss = 0.0

        # Also, as opposed to previous examples, we will be
        # uses "batches" of our data here in the gradient
        # descent training. By using the batches loaded from the data
        # loader, we can increase our training efficience (which
        # is important now that our model is much more
        # complicated). Otherwise, this is the very same thing
        # we have been doing since the first lessons.
        for i, data in enumerate(trainloader, 0):

            # get the inputs
            features, labels = data
            
            # NOTE - THIS IMPORTANT CHANGE
            features, labels = features.to(device), labels.to(device)

            # clear the gradients
            optimiser.zero_grad()

            # calculate the predicted values
            predictions = model.forward(features)

            # calculate our loss
            loss = criterion(predictions, labels)

            # implement our gradient-based updates to our
            # parammeters (putting them "back" into the model
            # via a "backward" update)
            loss.backward()
            optimiser.step()

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:
                print('[%d, %5d] loss: %.3f' %
                        (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0

    return model

# Connect to GPU

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# Run Training and Testing

In [8]:
# import and pre-process the CIFAR10 data
trainloader, testloader, classes = load_cifar10_dataset()

Files already downloaded and verified


In [10]:
# train our model on the GPU
model = nn_fit(trainloader, 0.001, 2, device)

[1,  2000] loss: 2.235
[1,  4000] loss: 1.893
[1,  6000] loss: 1.695
[1,  8000] loss: 1.559
[1, 10000] loss: 1.503
[1, 12000] loss: 1.458
[2,  2000] loss: 1.364
[2,  4000] loss: 1.339
[2,  6000] loss: 1.331
[2,  8000] loss: 1.320
[2, 10000] loss: 1.300
[2, 12000] loss: 1.248


In [13]:
# test the model
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy: %d %%' % (100 * correct / total))

Accuracy: 56 %


-----


# Data Science Ethics Checklist

## A. Data Collection
 - [ ] **A.1 Informed consent**: If there are human subjects, have those subjects have given informed consent, where users clearly understand what they are consenting to and there was a mechanism in place for gathering consent?
 - [ ] **A.2 Collection bias**: Have we considered sources of bias that could be introduced during data collection and survey design and taken steps to mitigate those?
 - [ ] **A.3 Limit PII exposure**: Have we considered ways to to minimize exposure of personally identifiable information (PII) for example through anonymization or not collecting information that isn't relevant for analysis?

## B. Data Storage
 - [ ] **B.1 Data security**: Do we have a plan to protect and secure data (e.g., encryption at rest and in transit, access controls on internal users and third parties, access logs, and up-to-date software)?
 - [ ] **B.2 Right to be forgotten**: Do we have a mechanism through which an individual can request their personal information be removed?
 - [ ] **B.3 Data retention plan**: Is there a schedule or plan to delete the data after it is no longer needed?

## C. Analysis
 - [ ] **C.1 Missing perspectives**: Have we sought to address blindspots in the analysis through engagement with relevant stakeholders (e.g., checking assumptions and discussing implications with affected communities and subject matter experts)?
 - [ ] **C.2 Dataset bias**: Have we examined the data for possible sources of bias and taken steps to mitigate or address these biases (e.g., stereotype perpetuation, confirmation bias, imbalanced classes, or omitted confounding variables)?
 - [ ] **C.3 Honest representation**: Are our visualizations, summary statistics, and reports designed to honestly represent the underlying data?
 - [ ] **C.4 Privacy in analysis**: Have we ensured that data with PII are not used or displayed unless necessary for the analysis?
 - [ ] **C.5 Auditability**: Is the process of generating the analysis well documented and reproducible if we discover issues in the future?

## D. Modeling
 - [ ] **D.1 Proxy discrimination**: Have we ensured that the model does not rely on variables or proxies for variables that are unfairly discriminatory?
 - [ ] **D.2 Fairness across groups**: Have we tested model results for fairness with respect to different affected groups (e.g., tested for disparate error rates)?
 - [ ] **D.3 Metric selection**: Have we considered the effects of optimizing for our defined metrics and considered additional metrics?
 - [ ] **D.4 Explainability**: Can we explain in understandable terms a decision the model made in cases where a justification is needed?
 - [ ] **D.5 Communicate bias**: Have we communicated the shortcomings, limitations, and biases of the model to relevant stakeholders in ways that can be generally understood?

## E. Deployment
 - [ ] **E.1 Redress**: Have we discussed with our organization a plan for response if users are harmed by the results (e.g., how does the data science team evaluate these cases and update analysis and models to prevent future harm)?
 - [ ] **E.2 Roll back**: Is there a way to turn off or roll back the model in production if necessary?
 - [ ] **E.3 Concept drift**: Do we test and monitor for concept drift to ensure the model remains fair over time?
 - [ ] **E.4 Unintended use**: Have we taken steps to identify and prevent unintended uses and abuse of the model and do we have a plan to monitor these once the model is deployed?

*Data Science Ethics Checklist generated with [deon](http://deon.drivendata.org).*
